-------------------------
#### prompt types
-----------------------

- Zero Shot prompting
    - we are using a pre-trained model
    - GPT 3.5
- Text Generation
- Question Answering (QnA)
- Summarization
- LT

In [2]:
import openai
from openai import OpenAI
import json

In [3]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    # api_key = openai_api_key
)

#### define a function for QnA

In [4]:
def question_answering(question, context):
    prompt = f"Q: {question}\nContext: {context}\nA:"

    response = client.chat.completions.create(
            model    = 'gpt-3.5-turbo',
            messages = [
                {"role" : "system", "content": "You are a helpful expert"},
                {"role" : "user",   "content": f"{prompt}"}
            ],
            max_tokens  = 500,
            temperature = 0.9,
            n           = 1
    )

    # extract the the answer from the response
    answer = response.choices[0].message.content

    return answer

Example - 01

In [5]:
context = '''patient seems to be stable but 5 days ago she suspected highly recurrent symptoms'''

In [6]:
question = '''Is the patient's condition stable presently" with a label prefix "Answer" '''

In [7]:
response = question_answering(question, context)
print(response)

Answer: The patient's condition is currently stable, although it is noted that she experienced highly recurrent symptoms 5 days ago. It would be important to monitor her closely to ensure ongoing stability.


Ex - 02

In [8]:
context = '''positive for TTF 1, and napsin A, suggestive of lung adenocarcinoma. 
Molecular testing was performed, that did not reveal actionable therapeutically targetable mutations. 
She underwent chemotherapy and radiation therapy, to this paraspinal mass, administered concurrently 
with carboplatin and paclitaxel. She was then treated with 4 cycles of systemic therapy with carboplatin, 
and pemetrexed, and then'''

question = '''Did the patient undergo radiation therapy? Your answer should be either 
'Yes' or 'NO' or "Not sure" with a label prefix "Answer" '''

In [9]:
response = question_answering(question, context)
print(response)

Answer: Yes


Ex - 03

In [10]:
context = '''consistent with parenchymal and potential leptomeningeal metastatic disease. 
There was no vasogenic edema shift or effect. A PET-CT scan also showed evidence of 
progression of disease. She met with Radiation Oncology, who has recommended radiation 
therapy to the brain, in the form of gamma knife therapy. She is consulting with other doctors if she really needs 
to undergo the radiation therapy
'''

question = '''Did the patient undergo radiation therapy ever? Your answer should be either 
'Yes' or 'NO' or "Not sure" with a label prefix "Answer" 

Also display "Explanation : " after displaying output of the question
'''

response = question_answering(question, context)
print(response)

Answer: Yes

Explanation: The patient has been recommended radiation therapy to the brain in the form of gamma knife therapy, as mentioned in the medical report.


Example - 04

In [11]:
context = '''
11-12 o'clock position of left breast, 6 CFN. Enlarged axillary lymph nodes were noted. 
Her right breast and axilla were benign. A small 8 mm lesion was noted in the left hepatic 
lobe that appeared benign. A chest CT scan performed that same day showed enlarged 
lymph nodes in the left axilla, no lytic or blastic disease within the bones, 
and an 8 mm benign appearing hemangioma in the liver. 

On 6/24/2020 left breast ultrasound biopsy was performed. 
This showed infiltrating ductal carcinoma that was grade 3 and triple negative.
Several weeks ago she had negative staging abdominal/pelvic CT and bone scan. 

A small left hepatic lesion was found and was felt to be consistent with a hemangioma. 

She has had a Port-A-Cath placed.
1 again reviewed and discussed extensively with the patient, and with her husband, 
her course to date. 

We outlined her node positive disease and probability that she has TNBC. 
We discussed the recommendation of neoadjuvant, and within this context discussed 
standard neoadjuvant chemotherapy, including traditional dose-dense AC followed by weekly paclitaxel, 
neoadjuvant chemo immunotherapy per the KEYNOTE-522 trial, and an open clinical trial, 
NSABP B-59. 

For various reasons, she does not wish to pursue treatment on B-59. 

We discussed the potential of delivering chemotherapy plus pembrolizumab per the KEYNOTE-522 trial, 
pending insurance approval. We reviewed potential side effects of chemotherapy including, 
but not limited to cytopenias, nausea, alopecia, and neuropathy. 

We discussed possible untoward effects of immunotherapy including autoimmune diseases. 
Through this discussion she provided consent to proceed.
'''

In [14]:
question = '''List out the names of the medications prescribed to the patients, in a bulleted text form
in the format below

Name of the medication: 
Dosage :
Frequency :
'''

response = question_answering(question, context)
print(response)

- Name of the medication: AC (Doxorubicin/Cyclophosphamide)
  Dosage: Standard dose
  Frequency: Every 2-3 weeks

- Name of the medication: Paclitaxel
  Dosage: Weekly
  Frequency: Weekly

- Name of the medication: Pembrolizumab
  Dosage: Per the KEYNOTE-522 trial
  Frequency: Pending insurance approval


In [28]:
questions_list = [
    'What is the diagnosis found in the left breast after the biopsy?',
    'Where was the primary lesion located in the patient’s left breast?',
    'What findings were noted in the left axillary region?',
    'What was the result of the chest CT scan performed on the same day?',
    'What were the findings regarding the small lesion in the liver?'
]

#### Sentiment Analysis

- classification (positive or negative or neutral)

In [36]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    
    messages = [{"role": "user", "content": prompt}]
    
    response = client.chat.completions.create(
        model      = model,
        messages   = messages,
        temperature= 0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [37]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

In [38]:
prompt = f"""

What is the sentiment of the following product review, which is delimited with triple backticks?

Review text: '''{lamp_review}'''

"""

In [39]:
response = get_completion(prompt)
print(response)

The sentiment of the review is positive.


Ex - 02

In [40]:
context = """
This film should be brilliant. It sounds like a great plot, the actors are first grade, and 
the supporting cast is good as well, and Stallone is attempting to deliver a good performance. 
However, it can’t hold up.
"""

prompt = f"""

What is the sentiment of the  review, which is delimited with triple backticks?

Review text: '''{context}'''

"""

response = get_completion(prompt)
response

'The sentiment of the review is disappointment.'

In [41]:
context = """
How can anyone sit through this movie?
"""

prompt = f"""

What is the sentiment of the following product review, which is delimited with triple backticks?

Review text: '''{context}'''

"""

response = get_completion(prompt)
print(response)

The sentiment of the review is negative.


In [42]:
context = """
My flight’s been delayed. Brilliant! 

"""

prompt = f"""

What is the sentiment of the following product review, which is delimited with triple backticks?

Review text: '''{context}'''

"""

response = get_completion(prompt)
print(response)

The sentiment of the review is negative.


In [43]:
context = """
My flight’s been delayed. It is great for me as I am running late! 

"""

prompt = f"""

What is the sentiment of the following product review, which is delimited with triple backticks?

Review text: '''{context}'''

"""

response = get_completion(prompt)
print(response)

The sentiment of the review is positive.


In [44]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


In [45]:
review = """

The movie story was unpredictable

"""

prompt = f"""
What is the sentiment of the following movie review, 
which is delimited with triple backticks?

Review text: '''{review}'''
"""

response = get_completion(prompt)
print(response)

The sentiment of the movie review is positive, as the reviewer found the story unpredictable.


In [46]:
review = """

The steering of the car is unpredictable. 

"""

prompt = f"""
What is the sentiment of the following vehicle review, 
which is delimited with triple backticks?

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the review is negative.


In [47]:
review = """

He thinks he is a Satyawadi Harishchandra. 

"""

prompt = f"""
What is the sentiment of the following review, which is delimited with triple backticks? 

State the output as Sentiment : <output>

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

Sentiment : Neutral


In [49]:
review = """

He thinks he is a Gandhi. 

"""

prompt = f"""
What is the sentiment of the following review, which is delimited with triple backticks? 

State the output as Sentiment : <output>

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

Sentiment: Negative


In [50]:
review = """

I hate love stories.

"""

prompt = f"""
What is the sentiment of the following review, which is delimited with triple backticks? 

State the output as Sentiment : <output>

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

Sentiment : Negative


In [51]:
review = """

I love violent stories.

"""

prompt = f"""
What is the sentiment of the following review, which is delimited with triple backticks? 

State the output as Sentiment : <output>

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

Sentiment : Positive


In [53]:
review = """

I love communal politics.

"""

prompt = f"""
What is the sentiment of the following review, which is delimited with triple backticks? 

State the output as Sentiment : <output>

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

Sentiment : Positive
